<a href="https://colab.research.google.com/github/Matthew-Dickson/COS801Assignment/blob/main/COS801Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Some usefull information:</h2>


1.   https://www.kaggle.com/code/hsankesara/news-classification-using-han/notebook
2.   https://medium.com/analytics-vidhya/hierarchical-attention-networks-d220318cf87e



<h2>Dataset section <h2>